In [1]:
import os
os.chdir("D:\pp\Price-Analysis-DataScience")

In [20]:
from decouple import config

MLFLOW_TRACKING_URI = config('MLFLOW_TRACKING_URI')
MLFLOW_TRACKING_USERNAME= config('MLFLOW_TRACKING_USERNAME')
MLFLOW_TRACKING_PASSWORD = config('MLFLOW_TRACKING_PASSWORD')

In [2]:
os.environ["MLFLOW_TRACKING_URI"] = MLFLOW_TRACKING_URI
os.environ["MLFLOW_TRACKING_USERNAME"] = MLFLOW_TRACKING_USERNAME
os.environ["MLFLOW_TRACKING_PASSWORD"] = MLFLOW_TRACKING_PASSWORD

In [12]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluateConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [13]:
from src.price_prediction.constants import *
from src.price_prediction.utils.common import read_yaml, create_directories, save_json


In [14]:
class Configuration:
    def __init__(self, 
    config_filepath = CONFIG_FILE_PATH,
    params_filepath = PARAMS_FILE_PATH,
    schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
        
    def get_model_evaluation_config(self):
        config = self.config.model_evaluation
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])
        return ModelEvaluateConfig(
            root_dir = Path(config.root_dir),
            test_data_path = Path(config.test_data_path),
            model_path = Path(config.model_path),
            metric_file_name = Path(config.metric_file_name),
            target_column = str(list(schema.keys())[0]),
            mlflow_uri = config.mlflow_uri
        )
          

In [15]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib


In [18]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluateConfig):
        self.config = config

    
    def eval_metrics(self,actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
     
        r2 = r2_score(actual, pred)
        return rmse,r2
    


    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]


        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme


        with mlflow.start_run():

            predicted_qualities = model.predict(test_x)

            (rmse, r2) = self.eval_metrics(test_y, predicted_qualities)
            
            # Saving metrics as local
            scores = {"rmse": rmse, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)


            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            


            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.sklearn.log_model(model, "model", registered_model_name="GradientBoostingRegressor")
            else:
                mlflow.sklearn.log_model(model, "model")

    


In [19]:
try:
    config = Configuration()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(model_evaluation_config)
    model_evaluation.log_into_mlflow()
except Exception as e:
    raise Exception(f"Failed to log metrics into mlflow: {e}")

[2024-04-12 00:05:00,395: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-04-12 00:05:00,397: INFO: common: yaml file: params.yaml loaded successfully]
[2024-04-12 00:05:00,402: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-04-12 00:05:00,404: INFO: common: created directory at: artificats]
[2024-04-12 00:05:00,404: INFO: common: created directory at: artifacts\model_evaluation]
[2024-04-12 00:05:02,107: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]


c:\Users\vrush\anaconda3\envs\datascience\lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Successfully registered model 'GradientBoostingRegressor'.
2024/04/12 00:05:20 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: GradientBoostingRegressor, version 1
Created version '1' of model 'GradientBoostingRegressor'.
